In [4]:
import cv2
import os
import numpy

# Loading training data for face recognition

In [2]:
def read_images(path, image_size):
    names = []
    training_images, training_labels = [], []
    label = 0
    for dirname, subdirnames, filenames in os.walk(path):
        for subdirname in subdirnames:
            names.append(subdirname)
            subject_path = os.path.join(dirname, subdirname)
            for filename in os.listdir(subject_path):
                img = cv2.imread(os.path.join(subject_path, filename),
                                 cv2.IMREAD_GRAYSCALE)
                if img is None:
                    # The file cannot be loaded as an image.
                    # Skip it.
                    continue
                img = cv2.resize(img, image_size)
                training_images.append(img)
                training_labels.append(label)
            label += 1
    training_images = numpy.asarray(training_images, numpy.uint8)
    training_labels = numpy.asarray(training_labels, numpy.int32)
    return names, training_images, training_labels

This function walks through a directory's subdirectories, `loads the images`, `resizes` them to a specified size, and `puts` the resized images `in a list`. Then converts the lists into NumPy arrays.

It also builds `two` other lists: first, `a list of people's names or initials` and `a list of labels` associated with the loaded images.

In [ ]:
path_to_training_images = '/training'
training_image_size = (200, 200)
names, training_images, training_labels = read_images(
    path_to_training_images, training_image_size)

The function has `two input attributes`:
- `path_to_training_images`: Root path to the images that will be used for training.
- `training_image_size`: Specifies the size of the picked image.

In [ ]:
model = cv2.face.EigenFaceRecognizer_create()
model.train(training_images, training_labels)

We can create and train a face recognizer with these lines of code. (We test it with `EigenFaceRecognizer`)

`EigenFaceRecognizer_create()` function has two optional attributes:
- `num_components`: This is the number of components to keep for the PCA (`Principal Component Analysis`)
- `threshold`: This is a decimal number that indicates a confidence threshold. Images with values below threshold will be discarded.

We can test this by using a videocamera feed.

In [ ]:
face_cascade = cv2.CascadeClassifier(
    './cascades/haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0)
while (cv2.waitKey(1) == -1):
    success, frame = camera.read()
    if success:
        faces = face_cascade.detectMultiScale(frame, 1.3, 5)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            roi_gray = gray[x:x+w, y:y+h]
            if roi_gray.size == 0:
                # The ROI is empty. Maybe the face is at the image edge.
                # Skip it.
                continue
            roi_gray = cv2.resize(roi_gray, training_image_size)
            label, confidence = model.predict(roi_gray)
            text = '%s, confidence=%.2f' % (names[label], confidence)
            cv2.putText(frame, text, (x, y - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow('Face Recognition', frame)

For each detected face, we convert and resize it so we have a grayscale that matches the expected size (In our case, 200x200px)